# Understanding the `numpy.ndarray` internals

In [2]:
import numpy as np

In [3]:
x = np.array([[0, 1, 2, 3],[4, 5, 6, 7], [8, 9, 10, 11], [12, 13, 14, 15]], dtype=np.int8)
x

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]], dtype=int8)

***
### 1. Understanding strides
<mark>Question</mark> Determine the strides for the following arrays. Check your answer with `x.strides`.

In [4]:
y = x.reshape((2, 8))
y

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15]], dtype=int8)

In [5]:
# 1.2
z = x.reshape((1, 16))
z

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]],
      dtype=int8)

In [6]:
# 1.3
a = np.array([[0, 1, 2, 3],[4, 5, 6, 7],[8, 9, 10, 11], [12, 13, 14, 15]], dtype=np.int16)
a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [12, 13, 14, 15]], dtype=int16)

***
### 2. Metadata modification vs copying the data buffer

<mark>Question</mark> How do you explain the next result? Is it the same for `x.flatten()`?

In [7]:
x = np.random.rand(3, 3)
y = x.ravel()  #  flatten the array
y[0] = 100.
x

array([[1.00000000e+02, 7.74223986e-01, 7.04890216e-01],
       [3.64223228e-01, 5.66739243e-01, 5.13562598e-01],
       [2.46229952e-01, 2.28375250e-01, 6.02879382e-02]])

<mark>Question</mark> The next three cells do the same two operations: transposing a matrix and flattening it. How do you explain the difference in execution time?

In [19]:
x = np.random.rand(5000, 5000)

In [22]:
%%timeit
# 2.1
x.T
x.ravel()

455 ns ± 0.983 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [13]:
%%timeit
# 2.2
x.T.ravel()   #### ADD A FIGURE

128 ms ± 161 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
%%timeit
# 2.3
x.T
x.flatten()

73.5 ms ± 82.7 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Basics of broadcasting
The concept [broadcasting](https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html) has to do with the way NumPy treats the arrays during operations involving different shapes. For instance, an array of shape `(5,)` added to an escalar, gives an array of shape `(5, )` where to all the elements was added the escaler:

In [24]:
x = np.arange(5)   # of shape (5,)
x + 1.

array([1., 2., 3., 4., 5.])

An important operation in broadcasting is to create new dimensions of an array using `np.newaxis` .

In [25]:
x = np.arange(5)
x.shape

(5,)

In [26]:
x[:, np.newaxis].shape

(5, 1)

In [27]:
x[np.newaxis, :].shape

(1, 5)

<mark>Question</mark> From what you have already learned about the `numpy.ndarray`s, the operation `x[:, np.newaxis]` allocates new memory or can it be described with only a change on the metadata?

***

Broadcasting is often usefull to perform operations that are not vectorial in the mathematical sense, in a vectorial fashion. For instance, the next cell produces the array `y` with the different of all the possible combinations of the elements of `x`.

In [28]:
y = x[:, np.newaxis] + x[np.newaxis, :]
y.shape

(5, 5)

Here what happens is that each element of the `(5, 1)` array is added the `(5,)` element of the `(1, 5)` array. This will already know that produces an array of shape `(5,)`. Repeated for the five elements, this gives a `(5, 5)` array. 

***

Let's see how to get the difference of all combinations of the `(3,)` elements of a `(10, 3)` array:

In [29]:
x = np.random.rand(10, 3)
x.shape

(10, 3)

In [30]:
x[:, np.newaxis, :].shape

(10, 1, 3)

In [31]:
x[np.newaxis, :, :].shape

(1, 10, 3)

In [32]:
(x[np.newaxis, :, :] - x[:, np.newaxis, :]).shape

(10, 10, 3)

# Computing the Euclidean Distance Matrix with NumPy

The Eucledian distance matrix is an $n \times n$ matrix representing the spacing of a set of n points in Euclidean space. If $A$ is a Euclidean distance matrix and the points
$\mathbf{x}_{1}, \mathbf{x}_{2}, \ldots, \mathbf{x}_{n}$
are defined on $m$-dimensional space (each vector $\mathbf{x}$ has length $m$), then the elements of $A$ are given by
\begin{align}
A&=\left(a_{i j}\right);\\
a_{i j}&=d_{i j}^{2}=\left\|\mathbf{x}_{i}-\mathbf{x}_{j}\right\|^{2}
\end{align}
where $||.||^2$ denotes the 2-norm on $\mathbb{R}^m$.
\begin{equation}
A=\left[
\begin{array}{ccccc}
0 & d_{12}^{2} & d_{13}^{2} & \dots & d_{1 n}^{2} \\
d_{21}^{2} & 0 & d_{23}^{2} & \dots & d_{2 n}^{2} \\
d_{31}^{2} & d_{32}^{2} & 0 & \dots & d_{3 n}^{2} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
d_{n 1}^{2} & d_{n 2}^{2} & d_{n 3}^{2} & \dots & 0
\end{array}
\right]
\end{equation}

In this notebook we implement two functions to compute the Euclidean distance matrix. We use a simple algebra trick that makes possible to write the function in a completely vectorized way in terms of optimized NumPy functions.

In [37]:
import numpy as np
# Lets generate some random data
npoints = 10
ndimensions = 3
x = 10. * np.random.random([npoints, ndimensions])
y = 10. * np.random.random([npoints, ndimensions])

array([[6.67334609, 0.09632815, 6.43375331],
       [2.34816947, 7.4963108 , 1.33225589],
       [0.30789693, 8.91853404, 4.64525557],
       [2.86692177, 4.91585716, 2.183669  ],
       [9.67771591, 5.15586062, 5.96742542],
       [6.05339744, 5.20828354, 1.29178216],
       [2.52269482, 5.52159123, 6.68862814],
       [2.77543732, 9.1413536 , 4.39845428],
       [3.46202889, 4.09802567, 6.22671173],
       [8.43580322, 1.94185131, 9.25211586]])

In [41]:
"""Euclidean square distance matrix."""
diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]
dist = (diff * diff).sum(axis=2)

array([[124.04693558,   6.6128767 ,  28.50671185,  22.99347248,
         75.6416521 ,  21.56211358,  51.14109979,  14.20263853,
         56.98042588, 144.97716971],
       [127.13135347,   3.48753115,  13.02873318,  20.0957213 ,
         90.31359245,  31.21089171,  39.79460199,   7.72110166,
         49.53972274, 149.76463025],
       [ 59.22832587,  22.50722952,  53.21446025,   6.81785568,
         90.55362373,  23.23846547,  39.88749859,  52.53104052,
         31.16688014, 111.05576331],
       [ 24.04111227,  36.4712775 ,  46.63906036,  19.61866245,
         16.13044408,  18.78074118,  11.73033339,  28.5566611 ,
          5.90637709,  29.23163468],
       [ 89.83870048,  34.5131844 ,   8.60945687,  36.09223163,
         74.80351522,  62.01553368,   7.58810342,   9.90806764,
         20.11326991,  88.6563037 ],
       [ 26.19787362,  80.94214384, 139.92103871,  45.60398706,
         48.29718883,  27.10655769,  82.17281455, 111.83034144,
         55.74375248,  64.55028885],
       [ 6

<mark> Question </mark> At this point you are starting to get acquainted with the `numpy.ndarray`s and it's memory managment. Could you analyse the advantage and possible drawbacks of the `euclidean_broadcast` function?

***

Let's consider now a more sophisticated implementation:

In [44]:
"""Euclidean trick"""
x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]
xy = np.dot(x, y.T)
dist = np.abs(x2 + y2 - 2. * xy)

## The `euclidean_trick` function

Each element of the Euclidean distance matrix is the scalar product of the diference between two rows fo the dataset. `euclidean_trick` takes advantage of this by doing the following
$$\sum_k {(x_{ik}-y_{ik})^2} = (\vec{x}_i - \vec{y}_j)\cdot(\vec{x}_i - \vec{y}_j) = \vec{x}_i\cdot\vec{x}_i + \vec{y}_j\cdot\vec{y}_j - 2\vec{x}_i\cdot\vec{y}_j$$

There are NumPy functions to compute each of these terms:

$\vec{x}_i\cdot\vec{y}_j$ $\rightarrow$ `np.dot(x, y)` : Matrix product of $\{\vec{x}\}$ and $\{\vec{y}\}$

$\vec{x}_i\cdot\vec{x}_i$ $\rightarrow$ `np.einsum('ij,ij->i', x, x)[:, np.newaxis]` : A $(n,1)$ vector of elements $\sum_j x_{ij}x_{ij}$

$\vec{y}_j\cdot\vec{y}_j$ $\rightarrow$ `np.einsum('ij,ij->i', y, y)[np.newaxis, :]` : A $(1,n)$ vector of elements $\sum_j y_{ij}y_{ij}$

To have all the combinations $ij$ of the sum $\vec{x}_i\cdot\vec{x}_i + \vec{y}_j\cdot\vec{y}_j$, we add a new axis to each of the arrays, transpose one them and add them.

Let's see now how the `np.einsum` function works. `einsum` stands for Einstein summation, which is used in tensor algebra to write compact expressions without the sum symbol ($\sum$). Within the Einstein summation notation, whenever there are repeated indexes, there is a sum over them. For instance, the expression
$$x_{ik}y_{kj}$$
is equivalent to
$$\sum_k x_{ik}y_{kj}$$

`np.einsum` uses a generalized form of the Einstein summation by adding the symbol `->` to prevent summing over certain indexes. The specific operation we use here, `np.einsum('ij,ij->i', x, x)`, gives the vector
$$
\begin{bmatrix}
\sum_k x_{1k}x_{1k} \\
\sum_k x_{2k}x_{2k} \\
 ...                \\
\sum_k x_{nk}x_{nk} \\
\end{bmatrix}
$$
Note that the resulting vector is represented here as a column vector just for visualization purposes. It's is `(n,)` NumPy array.

Let's check now step-by-step what the `euclidean_trick` function does:

In [ ]:
# Lets generate some random data
nsamples = 10
nfeat = 3

x = 10. * np.random.random([nsamples, nfeat])

In [ ]:
x2 = np.einsum('ij,ij->i', x, x)
x2.shape

In [ ]:
x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
x2.shape

In [ ]:
(x2 + x2.T).shape

We now use `np.dot` to perform the matrix multiplication of the full dataset by itself. We didn't use it before as alternative to `np.einsum` because it doesn't perform row by row scalar products. Instead `np.dot` expects two arrays with matching shapes $(m,n)$ and $(n,m)$ to perform a matrix multiplication.

We could have used `np.einsum('ik,jk', x, x)` to perform the matrix multiplication, but we chose `np.dot(x, x.T)` instead. This is because `np.dot` is a very sophisticated too, plus it uses OpenMP threads. This results in a very fast execution.

You are wellcome to time them and look at the `top` command to see how `np.dot` uses multiple OpenMP threads.

In [ ]:
xy = np.dot(x, x.T)
xy.shape

Now, considering that the reason we are using `np.einsum` is to get rid of the loops, why didn't we use something like `(x*x).sum(axis=1)`? Let's run the next cell comparing them:

In [ ]:
# let's use a larger array for timing the function calls
nsamples = 1000
nfeat = 300

x = 10. * np.random.random([nsamples, nfeat])

# it gives the same result
np.abs(np.einsum('ij,ij->i', x, x) - (x*x).sum(axis=1)).max()

# but it's not as fast as `np.einsum`
%timeit np.einsum('ij,ij->i', x, x)
%timeit (x*x).sum(axis=1)

Doing a reduction with the ufunc `np.add` is also slower than `np.einsum`

In [ ]:
%timeit np.add.reduce(x*x, axis=1)

Finally, let's time both implementations and check that they give the same result!

In [ ]:
def euclidean_broadcast(x, y):
    diff = x[:, np.newaxis, :] - y[np.newaxis, :, :]
    return (diff * diff).sum(axis=2)

In [ ]:
def euclidean_trick(x, y):
    x2 = np.einsum('ij,ij->i', x, x)[:, np.newaxis]
    y2 = np.einsum('ij,ij->i', y, y)[np.newaxis, :]
    xy = np.dot(x, y.T)
    return np.abs(x2 + y2 - 2. * xy)

In [ ]:
nsamples = 2000
nfeat = 50

x = 10. * np.random.random([nsamples, nfeat])

%timeit euclidean_broadcast(x, x)
%timeit euclidean_trick(x, x)

In [ ]:
np.abs(euclidean_broadcast(x, x) - euclidean_trick(x, x)).max()

<mark> Question </mark> Change the implementation of `euclidean_broadcast` function to make faster using `einsum` to do the final sum. How much is the speed up? Compare it with both the original `euclidean_broadcast` and `euclidean_trick`. Check that the result is the same!

# Conclusions

The main points to take from this notebook are:
  * NumPy is all about vectorization. Loops in python must be avoided.
  * Always consider different vectorized implementations and compare them.
  * Even within NumPy, some functions might bring a more significant speedup than others.